In [ ]:
from sqlalchemy import or_, func
from sqlalchemy.orm import sessionmaker
from HardwareSwap.Models import Base, engine, Post, PostType, get_or_create, Series, Manufacturer, Brand, GPU, PostBrand, PostManufacturer
import tqdm

In [ ]:
recreate = True
if recreate:
    Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)
Session = sessionmaker(bind = engine)

In [ ]:
s = Session()
buying = get_or_create(s, PostType, post_type="buying")
selling = get_or_create(s, PostType, post_type="selling")
trading = get_or_create(s, PostType, post_type="trading")
giveaway = get_or_create(s, PostType, post_type="giveaway")
meta = get_or_create(s, PostType, post_type="meta")
official = get_or_create(s, PostType, post_type="official")
alert = get_or_create(s, PostType, post_type="alert")


In [ ]:
%load_ext autoreload
%autoreload 1
%aimport HardwareSwap
%aimport HardwareSwap.DownloadData
%aimport HardwareSwap.DownloadData.download_data
from HardwareSwap.DownloadData import download_data, remove_duplicate_rows
import os
import datetime
import pytz
import pandas as pd
from HardwareSwap.Models.Post import get_regex_to_parse_title
import re
import json

In [ ]:
data_root = "/home/neil/hardwareswap/raw_data/"
pcpartpicker_gpu_dir = os.path.join(data_root, "pcpartpicker_gpu")
raw_data = download_data.load_dataframe_from_disk(os.path.join(data_root, "reddit_posts", "data_*-*.json"), limit=5000)

print(f"Found {len(raw_data)} posts")
Post.create_bulk(raw_data, s)
Post.clean(s)

In [ ]:
s.rollback()

In [ ]:

pcpartpicker_gpus_fname = os.path.join(pcpartpicker_gpu_dir, "pcpartpicker_gpu.json")
if os.path.isfile(pcpartpicker_gpus_fname):
    with open(pcpartpicker_gpus_fname, 'r') as fp:
        gpus_all = json.load(fp)

In [ ]:
pcpartpicker_gpus_fname

In [ ]:
df = pd.DataFrame(gpus_all)
df["chipset"] = df.apply(lambda x: x["chipset"].replace("Chipset",""), axis=1)
df["brand"] = ""
df["mfg"] = ""

# set the brands
df.at[df["chipset"].str.contains("GeForce"), "brand"] = "nvidia"
df.at[df["chipset"].str.contains("Quadro"), "brand"] = "nvidia"
df.at[df["chipset"].str.contains("NVS"), "brand"] = "nvidia"
df.at[df["chipset"].str.contains("RTX"), "brand"] = "nvidia"
df.at[df["chipset"].str.contains("Titan"), "brand"] = "nvidia"
df.at[df["chipset"].str.contains("Radeon"), "brand"] = "amd"
df.at[df["chipset"].str.contains("FirePro"), "brand"] = "amd"
df.at[df["chipset"].str.contains("Vega"), "brand"] = "amd"
df.at[df["chipset"].str.contains("FireGL"), "brand"] = "amd"

# manufacture
brands = ["Sapphire","Asus","Gigabyte","EVGA","MSI","PNY","Zotac", "Lenovo", "GALAX","Inno3D","Jaton Video",
          "Galaxy","Sparkle", "ECS","Palit","KFA2" ,"OcUK","Gainward","XFX","BFG","NVIDIA","HP","Leadtek",
          "Zogis","Colorful","Biostar","Corsair","Dell", "HIS", "PowerColor", "VisionTek","Diamond", "Club 3D", "ASRock", "ATI","AMD", "Yeston"]
for brand in brands:
    df.at[df["name"].str.contains(brand), "mfg"] = brand.lower()


# Extract information about the gpus
result_names=["series","prefix","number","variant"]
for col in result_names:
    df[col] = ""
f_geforce = re.compile("(?P<series>(?:GeForce))?\s?(?P<prefix>(?:GTX)|(?:RTX)|(?:GT)|(?:GTS))\s?(?P<number>[123]?[0245679][1-9]0)\s?(?P<variant>(?:Ti))?")
f_quadro = re.compile("(?P<series>(?:Quadro))?\s?(?P<prefix>[KMP])\s?(?P<number>[12345678][02]0{2})\s?(?P<variant>(?:D))?")
f_titan = re.compile("(?P<series>(?:GeForce))?\s?(?P<prefix>(?:GTX)|(?:RTX))\s?(?P<number>(?:Titan))\s?(?P<variant>(?:Z)|(?:X)|(?:Black))?")
f_radeon = re.compile("(?P<series>(?:Radeon Pro)|(?:Radeon))\s?(?P<prefix>(?:RX)|(?:R[579])|(?:HD)|(?:WX))?\s?(?P<number>(?:[34567]?[123456789][03456789][05])|(?:Fury)|(?:VEGA [56][46])|(?:Nano)|(?:VII))?\s?(?P<variant>(?:X2)|(?:XT)|(?:X))?")
f_firepro = re.compile("(?P<series>(?:FirePro)|(?:FireGL))\s?(?P<prefix>(?:[VWRS][G]?))?\s?(?P<number>(?:[2345789]?[012346789][02567]0))\s?(?P<variant>(?:A?))?")
f_vega = re.compile("(?P<series>(?:Vega))\s?(?P<prefix>)?\s?(?P<number>)\s?(?P<variant>(?:Frontier Edition Liquid)|(?:Frontier Edition))")

for idx in range(len(df)):
    chipset = df.at[idx, "chipset"]
    for regex_function in [f_geforce, f_quadro, f_titan, f_radeon, f_firepro, f_vega]:
        match = regex_function.match(chipset)
        if match is None:
            continue
        series,prefix,number,variant = match.group(*result_names)
        df.at[idx,"series"] = series if not None else ""
        df.at[idx,"prefix"] = prefix if not None else ""
        df.at[idx,"number"] = number if not None else ""
        df.at[idx,"variant"] = variant if not None else ""
        break

#df[(df["brand"]=="nvidia") & (df["model"]=="")]
df[(df["brand"]=="amd" ) & (df["number"]=="")]


In [ ]:
def geforce_series(row):
    if row["series"] != "GeForce":
        return ""
    if not row["number"].isnumeric():
        return ""
    scale = 100
    series = f"GeForce {int(int(row['number'])/scale)}"
    return series

def get_vendor_series(row):
    if row["series"] == "GeForce":
        return geforce_series(row)

df["Vendor Series"] = df.apply(lambda row: geforce_series(row), axis=1)

series = {}
for item in df["Vendor Series"].unique():
    if item == "":
        continue
    elif item is None:
        continue
    series[item] = get_or_create(s,  Series, series=item)
    
mfgs = {}
for item in df["mfg"].unique():
    if item == "":
        continue
    elif item is None:
        continue
    mfgs[item] = get_or_create(s,  Manufacturer, manufacturer=item)

brands = {}
for item in df["brand"].unique():
    if item == "":
        continue
    elif item is None:
        continue
        
    brands[item] = get_or_create(s,  Brand, brand=item)
    

In [ ]:
s.rollback()
gpus = []
for idx in range(len(df)):
    row = df.iloc[idx]
    price = None
    if row["price"] != "0":
        price = float(row["price"])
    try:
        gpu = GPU(
            name = row["name"],
            brand_id = brands[row["brand"]].id,
            manufacturer_id = mfgs[row["mfg"]].id,
            series_id = series[row["Vendor Series"]].id,
            chipset = row["chipset"],
            prefix = row["prefix"],
            number = int(row["number"]),
            pcpartpicker_url = row["url"],
            pcpartpicker_price = price,       
                )
        gpus.append(gpu)
    except KeyError:
        pass
s.bulk_save_objects(gpus)
s.commit()
    

In [ ]:
s.rollback()

In [ ]:
post_brand = []

for brand in s.query(Brand):
    post_ids = s.query(Post.id).filter(func.lower(Post.selftext).contains(brand.brand.lower())).all()
    post_brand += [PostBrand(post_id=post_id[0], brand_id=brand.id) for post_id in post_ids]
    
s.bulk_save_objects(post_brand)
s.commit()

In [ ]:
post_mfg = []
for mfg in s.query(Manufacturer):
    post_ids = s.query(Post.id).filter(func.lower(Post.selftext).contains(mfg.manufacturer.lower())).all()
    post_mfg += [PostManufacturer(post_id=post_id[0], manufacturer_id=mfg.id) for post_id in post_ids]
    
s.bulk_save_objects(post_mfg)
s.commit()

In [ ]:

for items in s.query(Manufacturer.manufacturer, Post.title, Post.selftext).join(PostManufacturer, Post.id==PostManufacturer.post_id).join(Manufacturer, PostManufacturer.manufacturer_id==Manufacturer.id).filter(Manufacturer.manufacturer=="nvidia").limit(20).all():
    print()
    print(items)

In [ ]:
s.query(PostBrand.id).join(Brand, Brand.id==PostBrand.brand_id).filter(Brand.brand=="nvidia").count()

In [ ]:
import matplotlib.pyplot as plt
import re

In [ ]:
with open("/usr/share/dict/words",'r') as fp:
    words = fp.readlines()
words = [word.strip().lower() for word in words]
words = set(words)

In [ ]:
all_text = set()
counts = []
for idx, text in enumerate(s.query(func.lower(Post.selftext)).filter(Post.selftext!=None).all()):
    text = text[0]
    text = text.lower()
    text = re.sub('[^a-zA-Z0-9]', ' ', text)
    text = [item for item in text.split() if item.isalpha()]
    text = set(text)
    text = text.intersection(words)
    
    all_text.update(text)
    counts.append(len(all_text))
plt.plot(counts)
len(all_text)

In [ ]:
sorted(list(all_text))

In [ ]:
intersection = all_text.intersection(words)
print(f"Have {len(all_text)} words in posts, and {len(words)} words in dict")
print(f"Have {len(intersection)}")
